### Instalar dependencias e importarlas

In [5]:
!pip install -q mediapipe opencv-python pyarrow pandas

In [9]:
import os, math, json
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import pyarrow as pa, pyarrow.parquet as pq
import matplotlib.pyplot as plt
import glob

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


### Montar drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Configurar carpetas de acceso a drive

In [ ]:
DRIVE_VIDEOS = "/content/drive/MyDrive/proyecto/videos"   # aquí van los .mp4/.avi para la muestra
DRIVE_POSES  = "/content/drive/MyDrive/proyecto/poses"    # aquí se guardarán .parquet y resumen

os.makedirs(DRIVE_VIDEOS, exist_ok=True)
os.makedirs(DRIVE_POSES,  exist_ok=True)

###Recolectar los videos a procesar

In [ ]:
VIDEO_EXTS = ("*.mp4","*.avi","*.mov","*.mkv","*.wmv","*.flv")
video_paths = []
for ext in VIDEO_EXTS:
    video_paths += glob.glob(os.path.join(DRIVE_VIDEOS, ext))
video_paths = sorted(video_paths)

print(f"🎬 Videos encontrados: {len(video_paths)}")
for p in video_paths[:5]:
    print("  -", os.path.basename(p))
if not video_paths:
    print("No hay videos en", DRIVE_VIDEOS)


### Detección (SMOKE TESTS)

Muestra 1 frame de los primeros N videos ajustados en la variable N_SMOKE

In [3]:
from google.colab.patches import cv2_imshow

N_SMOKE = 1  # cambia a 2-3 si quieres ver más

for i, vp in enumerate(video_paths[:N_SMOKE]):
    cap = cv2.VideoCapture(vp)
    ok, frame = cap.read()
    cap.release()
    if not ok:
        print(f"⚠️ No pude leer: {os.path.basename(vp)}"); continue

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = pose.process(rgb)
        bgr = frame.copy()
        if res.pose_landmarks:
            mp_drawing.draw_landmarks(bgr, res.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            print(f"✅ Smoke test OK en: {os.path.basename(vp)}")
        else:
            print(f"⚠️ Sin detección en frame inicial: {os.path.basename(vp)}")
        cv2_imshow(bgr)


### Guardar landmarks por frame

In [ ]:
import math
import numpy as np
import pyarrow as pa, pyarrow.parquet as pq
import mediapipe as mp, cv2

def extract_video_to_parquet(in_video: str, out_parquet: str,
                             stride: int = 1,
                             model_complexity: int = 1,
                             min_det: float = 0.5,
                             min_trk: float = 0.5) -> dict:

    mp_pose = mp.solutions.pose
    cap = cv2.VideoCapture(in_video)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    if fps == 0 or math.isclose(fps, 0.0):
        fps = 30.0
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    rows = []
    f = -1
    miss = 0
    LM = 33

    with mp_pose.Pose(model_complexity=model_complexity,
                      min_detection_confidence=min_det,
                      min_tracking_confidence=min_trk,
                      smooth_landmarks=True) as pose:

        while True:
            ok, frame = cap.read()
            if not ok:
                break
            f += 1
            if f % stride != 0:
                continue

            t = f / fps
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb.flags.writeable = False
            res = pose.process(rgb)

            x = np.full(LM, np.nan, np.float32)
            y = np.full(LM, np.nan, np.float32)
            v = np.zeros(LM, np.float32)

            if res.pose_landmarks:
                for i, lm in enumerate(res.pose_landmarks.landmark[:LM]):
                    x[i], y[i], v[i] = lm.x * W, lm.y * H, getattr(lm, "visibility", 0.0)
            else:
                miss += 1

            row = {"frame_idx": f, "t_sec": float(t), "width": W, "height": H, "fps": float(fps)}
            for i in range(LM):
                row[f"x_{i}"] = float(x[i])
                row[f"y_{i}"] = float(y[i])
                row[f"vis_{i}"] = float(v[i])
            rows.append(row)

    cap.release()
    pq.write_table(pa.Table.from_pylist(rows), out_parquet)

    return {
        "video": os.path.basename(in_video),
        "parquet": os.path.basename(out_parquet),
        "frames": len(rows),
        "miss_frames": miss,
        "fps": float(fps),
        "width": int(W),
        "height": int(H),
        "stride": int(stride),
        "model_complexity": int(model_complexity),
        "min_det": float(min_det),
        "min_trk": float(min_trk),
    }


### Procesar los videos y guardar un resumen en CSV

In [ ]:
import pandas as pd

summaries = []

if not video_paths:
    print("⚠️ No hay videos para procesar. Añade archivos a:", DRIVE_VIDEOS)
else:
    for vp in video_paths:
        base = os.path.splitext(os.path.basename(vp))[0]
        out_parquet = os.path.join(DRIVE_POSES, f"{base}.parquet")
        print(f"→ Procesando {os.path.basename(vp)} ...")
        info = extract_video_to_parquet(vp, out_parquet,
                                        stride=1,
                                        model_complexity=1,
                                        min_det=0.5, min_trk=0.5)
        summaries.append(info)
        print(f"   ✔ Guardado: {os.path.basename(out_parquet)} | frames:{info['frames']} | miss:{info['miss_frames']}")

    # Guardar resumen
    summary_df = pd.DataFrame(summaries)
    summary_csv = os.path.join(DRIVE_POSES, "poses_summary.csv")
    summary_df.to_csv(summary_csv, index=False)
    print("\n📄 Resumen guardado en:", summary_csv)

summary_df.head() if len(summaries) else None


### Abrir un parquet generado

In [ ]:
# Carga el primero que se haya creado y mira columnas/filas
generated_parquets = sorted(glob.glob(os.path.join(DRIVE_POSES, "*.parquet")))
print("Parquet generados:", len(generated_parquets))
if generated_parquets:
    test_pq = generated_parquets[0]
    print("Mostrando:", os.path.basename(test_pq))
    df = pd.read_parquet(test_pq)
    display(df.head())
    print(df.shape)


### Calcular angulos

In [ ]:
def calculate_angle(a,b,c):
    a=np.array(a); b=np.array(b); c=np.array(c)
    radians=np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180.0: angle=360-angle
    return angle
